In [111]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Загрузка данных

In [112]:
import joblib
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import seaborn as sns
import datetime
#!pip install -U https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

#!pip install -U pandas-profiling

import pandas_profiling as pp
from pandas_profiling import ProfileReport

In [113]:
model_initial = joblib.load("/content/drive/MyDrive/MTS.Project/Strategy/Trend_classification/trend_classification_model_GAZP.pkl")

In [114]:
df = pd.read_csv("/content/drive/MyDrive/MTS.Project/DataSet/GazNeft_H1/GAZP_H1_HLOCV_MTS_Project_4_8_2021___5-41-42.csv", sep=";")

In [115]:
df = df[['TIMEDATE','Close']]
df['TIMEDATE'] = pd.to_datetime(df['TIMEDATE'])

## Формирование датасета

In [116]:
df_dif = df[1:].reset_index(drop = True)
df_dif['Close'] = df[1:]['Close'].reset_index(drop = True,) - df[0:len(df)-1]['Close']

In [117]:
df_dif

,TIMEDATE,Close
0,2017-01-03 13:00:00,-0.22
1,2017-01-03 14:00:00,0.19
2,2017-01-03 15:00:00,0.44
3,2017-01-03 16:00:00,-0.01
4,2017-01-03 17:00:00,0.07
...,...,...
12918,2021-08-03 17:00:00,0.93
12919,2021-08-03 18:00:00,0.93
12920,2021-08-03 19:04:00,-0.14
12921,2021-08-03 20:00:00,-0.28


In [118]:
def trend_mark(df_dif):
  trend = 1
  trend_strength_list = []
  flag = True

  for i in tqdm(range(len(df_dif))):
    flag = True
    k = i
    if df_dif['Close'].iloc[i] == 0:
      trend_strength_list.append(0)
      continue
    while flag == True:
      try:
        if df_dif['Close'].iloc[i] * df_dif['Close'].iloc[k + 1] > 0:
          k += 1
          trend += 1
        else :
          trend_strength_list.append(trend)
          trend = 1
          flag = False
      except:
          trend_strength_list.append(trend)
          trend = 1
          flag = False

    trend_classified_strength_list=[]

  for iter in tqdm (trend_strength_list):
    if iter == 0 or iter == 1:
      trend_classified_strength_list.append(iter)
    else:
      trend_classified_strength_list.append(2)
  return trend_classified_strength_list
        
  
 


In [119]:
# trend_classified_strength_list=[]
# for iter in tqdm (trend_strength_list):
#   if iter == 0 or iter == 1:
#     trend_classified_strength_list.append(iter)
#   elif iter > 1 and iter < 4:
#     trend_classified_strength_list.append(2)
#   else:
#     trend_classified_strength_list.append(3)

In [120]:
trend_classified_strength_list=[]
for iter in tqdm (trend_strength_list):
  if iter == 0 or iter == 1:
    trend_classified_strength_list.append(iter)
  else:
    trend_classified_strength_list.append(2)

100%|██████████| 12923/12923 [00:00<00:00, 832675.18it/s]


In [121]:
trend_classified_strength_list = trend_mark(df_dif)

100%|██████████| 12923/12923 [00:00<00:00, 1873267.34it/s]


In [122]:
len(trend_classified_strength_list)

12923

In [123]:
df_dif['trend_strength'] = trend_classified_strength_list

In [124]:
df_dif['trend_strength'].value_counts()

1    6534
2    6245
0     144
Name: trend_strength, dtype: int64

Классы несбалансированы (особенно целевой класс, сильный тренд, находится в меньшенстве)

In [125]:
def make_dataset (data, hours = 10, vol_column = 'Volume', cost_column = 'Close', target_column = 'trend_strength'):
  
  d={}
  step = hours+1
  initial_start = (len(data))%(step) #чтобы избежать конфликта рамеров столбцов, уберем первые несколько строк из датасета


  d['start_datatime'] = list(data[initial_start::step]['TIMEDATE']) # взять 0 день, 11 день
  for i in range(hours):
    # В новом датасете будут колонки 1vol, 1cost, 2vol, 2cost, ..., 10vol, 10cost
    # Количество столбцов = hours + 1(таргет) +2(дата первого дня и полследнего)
    end = len(data) - hours + i     
    #d[str(i+1)+'vol'] = list(data[initial_start + i:end:step][vol_column]) # от вида среза (slice) зависит количесвто строк
    d[str(i+1)+'dif'] = list(data[initial_start + i:end:step][cost_column]) # каждая итерация цикла передаёт данные в один столбец

  
  d['end_datatime'] = list(data[initial_start + hours -1::step]['TIMEDATE'])  # дата последнего столбца в датасете
  d['target'] = list(data[initial_start  + hours::step][target_column]) # таргет
  d['target_datetime'] = list(data[initial_start  + hours::step]['TIMEDATE']) # таргет дата
  
  return pd.DataFrame(d)

In [126]:
df_new = make_dataset(df_dif)

In [127]:
df_new.head()

,start_datatime,1dif,2dif,3dif,4dif,5dif,6dif,7dif,8dif,9dif,10dif,end_datatime,target,target_datetime
0,2017-01-04 12:00:00,0.14,-0.13,-0.22,0.10,-0.22,0.19,1.04,0.00,0.30,-1.14,2017-01-05 11:00:00,2,2017-01-05 12:00:00
1,2017-01-05 13:00:00,-0.29,-0.73,-0.34,0.37,-0.53,0.30,0.15,0.08,-1.42,0.11,2017-01-06 12:00:00,2,2017-01-06 13:00:00
2,2017-01-06 14:00:00,0.02,0.25,-0.05,0.50,0.44,0.10,0.57,0.56,-1.02,0.00,2017-01-09 13:00:00,2,2017-01-09 14:00:00
3,2017-01-09 15:00:00,-0.15,-0.13,0.34,0.50,-0.39,1.16,0.18,0.25,0.37,-0.02,2017-01-10 14:00:00,2,2017-01-10 15:00:00
4,2017-01-10 16:00:00,1.28,-0.18,0.53,-0.47,-0.15,0.63,-0.27,-0.28,-0.38,-0.97,2017-01-11 15:00:00,1,2017-01-11 16:00:00


## Profile report

In [128]:
#profile = ProfileReport(df_new, title='Pandas Profiling Report')
#profile.to_file("Profiling_Report_Normalize_CII.html")

## Разделение на train / test

In [129]:
from sklearn.model_selection import train_test_split

In [130]:
from sklearn import preprocessing

In [164]:
X_train, X_test, y_train, y_test = train_test_split(df_new.drop(columns = ['start_datatime','end_datatime','target_datetime', 'target',]), df_new['target'], test_size=0.15, stratify = df_new['target'])
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, '/content/drive/MyDrive/MTS.Project/Strategy/Trend_classification/Normalizer/strength_classification_normalizer_GAZP.pkl')

['/content/drive/MyDrive/MTS.Project/Strategy/Trend_classification/Normalizer/strength_classification_normalizer_GAZP.pkl']

##Подбор моделей

In [132]:
from sklearn.metrics import  recall_score, precision_score, roc_curve

### Kneighbors

In [133]:
from sklearn.neighbors import KNeighborsClassifier


In [134]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train_scaled,y_train)
 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [135]:
sns.set(font_scale=1.5)
sns.set_color_codes("muted")

plt.figure(figsize=(10, 8))
fpr, tpr, thresholds = roc_curve(y_train, neigh.predict_proba(X_train_scaled)[:,1], pos_label=0)
lw = 2
plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig("ROC.png")
plt.show()

### Voting Classifier

In [136]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [137]:
clf1 = KNeighborsClassifier(n_neighbors = 3, metric='euclidean')
clf2 = MLPClassifier(hidden_layer_sizes = (200,),
                     learning_rate = 'constant',
                     
                     max_iter = 200)
clf3 = AdaBoostClassifier(n_estimators = 150)
clf4 = GradientBoostingClassifier(n_estimators=12, learning_rate=1.0, max_depth=1)

In [138]:
model  = VotingClassifier(estimators=[('KNN', clf1), ('MLP', clf2), ('Ada', clf3), ('GB', clf4)], voting='soft')
model.fit(X_train_scaled, y_train)
preds = (model.predict(X_test_scaled)>0.5).astype(int)
print (recall_score(y_test, preds, average='weighted'), precision_score(y_test, preds, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.5254237288135594 0.2776386748844376


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [139]:
sns.set(font_scale=1.5)
sns.set_color_codes("muted")

plt.figure(figsize=(10, 8))
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1], pos_label=1)
lw = 2
plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig("ROC.png")
plt.show()

In [140]:
from sklearn.metrics import multilabel_confusion_matrix

In [141]:
r = multilabel_confusion_matrix(y_test, model.predict(X_test), labels = [0, 1, 2, 3])

In [142]:
print(r)

[[[176   0]
  [  1   0]]

 [[ 42  42]
  [ 38  55]]

 [[ 56  38]
  [ 41  42]]

 [[177   0]
  [  0   0]]]


### SVC

In [143]:
from sklearn.svm import SVC

In [144]:
from sklearn.metrics import plot_confusion_matrix

In [145]:
clf = SVC(gamma='auto', class_weight = 'balanced', kernel = 'rbf')

In [146]:
clf.fit (X_train_scaled, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [147]:
plot_confusion_matrix(clf, X_test_scaled, y_test, labels= y_test.unique()) 

In [148]:
r = multilabel_confusion_matrix(y_test, clf.predict(X_test), labels = [0, 1, 2, 3])
print(r)

[[[168   8]
  [  1   0]]

 [[ 43  41]
  [ 40  53]]

 [[ 58  36]
  [ 44  39]]

 [[177   0]
  [  0   0]]]


### GRADBoost

In [149]:
from sklearn.ensemble import GradientBoostingClassifier

In [150]:
model = GradientBoostingClassifier()

In [151]:
model.fit(X_train_scaled, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [152]:
plot_confusion_matrix(model, X_test_scaled, y_test, labels= y_test.unique()) 

### ONE vs Rest

In [153]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier


In [154]:
plot_confusion_matrix(OneVsRestClassifier(SVC(random_state=0)).fit(X_train_scaled, y_train), X_test_scaled, y_test, labels = y_test.unique()) 
plot_confusion_matrix(OneVsRestClassifier(AdaBoostClassifier(n_estimators = 88, random_state=0)).fit(X_train_scaled, y_train), X_test_scaled, y_test, labels = y_test.unique()) 
plot_confusion_matrix(OneVsRestClassifier(GradientBoostingClassifier(n_estimators = 88, random_state=0)).fit(X_train_scaled, y_train), X_test_scaled, y_test, labels = y_test.unique()) 
plt.show()

Exception ignored in: <function tqdm.__del__ at 0x7f0c99e6fd40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1152, in __del__
    self.close()
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1281, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'


In [155]:
model = OneVsRestClassifier(AdaBoostClassifier(n_estimators = 88, random_state=0)).fit(X_train_scaled, y_train)

One vs One

In [156]:

from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

In [157]:
plot_confusion_matrix(OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train_scaled, y_train), X_test_scaled, y_test, labels = y_test.unique()) 
plot_confusion_matrix(OneVsOneClassifier(AdaBoostClassifier(n_estimators = 88, random_state=0)).fit(X_train_scaled, y_train), X_test_scaled, y_test, labels = y_test.unique()) 
plot_confusion_matrix(OneVsOneClassifier(SVC( random_state=0)).fit(X_train_scaled, y_train), X_test_scaled, y_test, labels = y_test.unique()) 

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## Экономическая оценка

In [158]:
df_dif

,TIMEDATE,Close,trend_strength
0,2017-01-03 13:00:00,-0.22,1
1,2017-01-03 14:00:00,0.19,2
2,2017-01-03 15:00:00,0.44,1
3,2017-01-03 16:00:00,-0.01,1
4,2017-01-03 17:00:00,0.07,2
...,...,...,...
12918,2021-08-03 17:00:00,0.93,2
12919,2021-08-03 18:00:00,0.93,1
12920,2021-08-03 19:04:00,-0.14,2
12921,2021-08-03 20:00:00,-0.28,1


In [57]:
answers = []
for i in tqdm(range (len(df_dif)-10)):
  answers.append(model.predict([df_dif['Close'].iloc[i:i+10]]))

100%|██████████| 12913/12913 [09:14<00:00, 23.29it/s]


In [58]:
len(answers)

12913

In [59]:
df_dif['trend_strength_preds'] = np.nan
df_dif['trend_strength_preds'].iloc[10:] = answers

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [60]:
answers_2 = []
for i in tqdm(range (len(df_dif)-10)):
  answers_2.append(model_initial.predict([df_dif['Close'].iloc[i:i+10]]))

100%|██████████| 12913/12913 [02:58<00:00, 72.45it/s]


In [61]:
df_dif['trend_direction'] = np.nan
df_dif['trend_direction'].iloc[10:] = answers_2

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [62]:
df_dif.head(50)

,TIMEDATE,Close,trend_strength,trend_strength_preds,trend_direction
0,2017-01-03 13:00:00,-0.22,1,NaN,NaN
1,2017-01-03 14:00:00,0.19,2,NaN,NaN
2,2017-01-03 15:00:00,0.44,1,NaN,NaN
3,2017-01-03 16:00:00,-0.01,1,NaN,NaN
4,2017-01-03 17:00:00,0.07,2,NaN,NaN
5,2017-01-03 18:00:00,0.38,1,NaN,NaN
6,2017-01-04 09:59:00,-0.86,2,NaN,NaN
7,2017-01-04 10:00:00,-0.44,1,NaN,NaN
8,2017-01-04 11:00:00,0.14,2,NaN,NaN
9,2017-01-04 12:00:00,0.14,1,NaN,NaN


In [63]:
df_dif['trend_direction'].value_counts()

0.0    7725
1.0    5188
Name: trend_direction, dtype: int64

In [64]:
df_dif.head(40)

,TIMEDATE,Close,trend_strength,trend_strength_preds,trend_direction
0,2017-01-03 13:00:00,-0.22,1,NaN,NaN
1,2017-01-03 14:00:00,0.19,2,NaN,NaN
2,2017-01-03 15:00:00,0.44,1,NaN,NaN
3,2017-01-03 16:00:00,-0.01,1,NaN,NaN
4,2017-01-03 17:00:00,0.07,2,NaN,NaN
5,2017-01-03 18:00:00,0.38,1,NaN,NaN
6,2017-01-04 09:59:00,-0.86,2,NaN,NaN
7,2017-01-04 10:00:00,-0.44,1,NaN,NaN
8,2017-01-04 11:00:00,0.14,2,NaN,NaN
9,2017-01-04 12:00:00,0.14,1,NaN,NaN


In [65]:
(df_dif[df_dif['TIMEDATE'].dt.date==pd.to_datetime('2017-01-06')])['trend_strength'].mean()

1.6

In [66]:
(df_dif[df_dif['TIMEDATE'].dt.date==pd.to_datetime('2017-02-20')])['trend_direction'].mean()

0.6

In [67]:
pd.to_datetime('2017-01-03')+datetime.timedelta(days=1)

Timestamp('2017-01-04 00:00:00')

In [68]:
df_dif['TIMEDATE'].dt.date.unique()

array([datetime.date(2017, 1, 3), datetime.date(2017, 1, 4),
       datetime.date(2017, 1, 5), ..., datetime.date(2021, 7, 30),
       datetime.date(2021, 8, 2), datetime.date(2021, 8, 3)], dtype=object)

In [69]:
df_final = pd.DataFrame({'DATE': df_dif['TIMEDATE'].dt.date.unique()})

In [70]:
df_final

,DATE
0,2017-01-03
1,2017-01-04
2,2017-01-05
3,2017-01-06
4,2017-01-09
...,...
1148,2021-07-28
1149,2021-07-29
1150,2021-07-30
1151,2021-08-02


In [71]:
list_1 = []
list_2 = []

for date in tqdm(df_dif['TIMEDATE'].dt.date.unique()):
  list_1.append((df_dif[df_dif['TIMEDATE'].dt.date==pd.to_datetime(date)])['trend_strength_preds'].mean())
  list_2.append((df_dif[df_dif['TIMEDATE'].dt.date==pd.to_datetime(date)])['trend_direction'].mean())
  

100%|██████████| 1153/1153 [00:13<00:00, 83.21it/s]


In [72]:
df_final['trend_strengh'] = list_1
df_final['trend_direction'] = list_2

In [73]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153 entries, 0 to 1152
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DATE             1153 non-null   object 
 1   trend_strengh    1152 non-null   float64
 2   trend_direction  1152 non-null   float64
dtypes: float64(2), object(1)
memory usage: 27.1+ KB


In [74]:
date_trend_list = []
bought = False
for index, line in df_final.iterrows():
  if line['trend_strengh']>1.5 and line['trend_direction']>0.5 and bought == False:
    buy_date = np.datetime64(line['DATE'])
    bought = True
  if line['trend_strengh']<=1.5 and line['trend_direction']<=0.5 and bought == True:
    sell_date = np.datetime64(line['DATE'])
    bought = False 
    date_trend_list.append([buy_date,sell_date])



In [75]:
date_trend_list

[[numpy.datetime64('2017-01-10'), numpy.datetime64('2017-01-11')],
 [numpy.datetime64('2017-02-20'), numpy.datetime64('2017-02-21')],
 [numpy.datetime64('2017-03-21'), numpy.datetime64('2017-03-22')],
 [numpy.datetime64('2017-03-30'), numpy.datetime64('2017-03-31')],
 [numpy.datetime64('2017-04-04'), numpy.datetime64('2017-04-05')],
 [numpy.datetime64('2017-04-11'), numpy.datetime64('2017-04-12')],
 [numpy.datetime64('2017-04-26'), numpy.datetime64('2017-04-28')],
 [numpy.datetime64('2017-05-12'), numpy.datetime64('2017-05-15')],
 [numpy.datetime64('2017-05-31'), numpy.datetime64('2017-06-01')],
 [numpy.datetime64('2017-07-07'), numpy.datetime64('2017-07-10')],
 [numpy.datetime64('2017-07-13'), numpy.datetime64('2017-07-14')],
 [numpy.datetime64('2017-07-17'), numpy.datetime64('2017-07-18')],
 [numpy.datetime64('2017-08-08'), numpy.datetime64('2017-08-10')],
 [numpy.datetime64('2017-08-15'), numpy.datetime64('2017-08-16')],
 [numpy.datetime64('2017-09-25'), numpy.datetime64('2017-09-26

In [87]:
days_trend = 0
for period in date_trend_list:
  days_trend += (period[1]-period[0])/ np.timedelta64(1, 'D')
days_trend/=4
days_trend

79.0

In [76]:
symbol_buy_sell_dates = {'GAZP':date_trend_list}

In [77]:
#df_.index[df_.index.get_loc(buy_date, method='nearest')]
symbols = ['GAZP']
df_ = pd.read_csv('/content/drive/MyDrive/MTS.Project/DataSet/GazNeft_D1/GAZP_D1_HLOCV_MTS_Project_7_8_2021___12-7-41.csv', sep=";", parse_dates=['TIMEDATE'], index_col='TIMEDATE')
symbols_df = {'GAZP': df_}

In [78]:
# перечень доходностей по символу
df_symbol_div_profit = {}
# Рассчет доходности (цена в дату закрытия минус цена на дату открытия)
for symbol in symbols:
    sum_profit = 0
    df_ = symbols_df[symbol]
    df_symbol_div_profit[symbol] = {}
    for bs_date in symbol_buy_sell_dates[symbol]:
        buy_date = bs_date[0]
        sell_date = bs_date[1]
        
        # Если искомая дата открытия сделки меньше чем самая минимальная доступная дата в датасете акций тогда пропускаем эту дату открытия и переходим к следующей. 
        # Аналогично если дата закрытия сделки выходит за перчень досутных дат акций то также пропускаем и переходим к следующей
        # Если дата открытия (buy_date) и дата закрытия (sell_date) находятся внутри диапозона дат в датасете акций тогда продолжаем выполнение кода
        if buy_date < df_.index.min() or sell_date > df_.index.max(): 
            continue
        
        # ищем ближайшую отсечку (дата свечи акции) к дате открытия и дате закрытия
        # Опредлеяем отсечку (дата свечи акции) которая ближайшая относительно даты открытия (не берется ближайшая следующая так как у нас таймфре2йм дневной и мы просто перелетим в другой день)
        buy_date = df_.index[df_.index.get_loc(buy_date, method='nearest')]
        # Опредлеяем отсечку (дата свечи акции) которая ближайшая относительно даты закрытия  (не берется ближайшая предыдущая так как у нас таймфре2йм дневной и мы просто перелетим в другой день)
        sell_date = df_.index[df_.index.get_loc(sell_date, method='nearest')]
        
        buy_close = df_.loc[buy_date]["Close"]
        sell_close = df_.loc[sell_date]["Close"]
        sum_profit += round(sell_close- buy_close, 2)
        # print("buy_date: ", buy_date, "sell_date: ", sell_date)
        # print("buy_close: ", buy_close, "sell_close: ", sell_close)
        # print("(sell_close- buy_close)", round(sell_close- buy_close,2))
        # print("Доходность: ", " Абсолютное значние: ", round(sell_close- buy_close, 2), " Относительное значние: ", round((sell_close- buy_close)/buy_close,4),"%"  )

        # Далее идет констуркция которая нужна для свода данных в одну стурктуру, вероятно лучше воспользоваться ООП для этого, но пока так реализовываем
        # Если за этот год уже есть данные о прибыльности для этого символа, тогда приплюсовываем новую и увеличиваем счетчик дивидендов на 1 опять же таки только для этого года
        # Счетчики дивидендов надо учитывать потому что у разных символа (акции) разное кол-во выплат в года у кого 1 у кого 2, но также и для одного символа может быть в разные года разное кол-во выплат, напримре у ROSN за 2020 год одна выплата а за лругие 2
        if f"count_div_{buy_date.year}" in df_symbol_div_profit[symbol]:
            df_symbol_div_profit[symbol][f"profit_div_{buy_date.year}"] += round(sell_close- buy_close, 2)
            df_symbol_div_profit[symbol][f"prc_profit_div_{buy_date.year}"] += round((sell_close- buy_close)/buy_close*100, 2)
            df_symbol_div_profit[symbol][f"count_div_{buy_date.year}"] += 1
        else:
            df_symbol_div_profit[symbol][f"profit_div_{buy_date.year}"] = round(sell_close- buy_close, 2)
            df_symbol_div_profit[symbol][f"prc_profit_div_{buy_date.year}"] = round((sell_close- buy_close)/buy_close*100, 2)
            df_symbol_div_profit[symbol][f"count_div_{buy_date.year}"] = 1
        
    print(f"{symbol} Доходность по Див.модели: ", round(sum_profit,2))
df_symbol_div_profit = pd.DataFrame.from_dict(df_symbol_div_profit, orient='index')
# Данные по доходности 
df_symbol_div_profit

GAZP Доходность по Див.модели:  22.08


,profit_div_2017,prc_profit_div_2017,count_div_2017,profit_div_2018,prc_profit_div_2018,count_div_2018,profit_div_2019,prc_profit_div_2019,count_div_2019,profit_div_2020,prc_profit_div_2020,count_div_2020,profit_div_2021,prc_profit_div_2021,count_div_2021
GAZP,4.7,3.55,21,10.76,7.43,4,-7.89,-4.38,5,12.79,6.82,11,1.72,0.82,4


In [89]:
# Выбираем только те колонки, которые содержат процент доходности
prc_profit_div_cols = [col for col in df_symbol_div_profit.columns if 'prc_profit_div' in col]
mean_prc_profit_div = round(df_symbol_div_profit[prc_profit_div_cols].mean().mean(),2)
print(f"Средняя доходность рассматриваемой модели (PM) за весь период (с {df_.index.min().year} по {df_.index.max().year}):", mean_prc_profit_div, "%")

Средняя доходность рассматриваемой модели (PM) за весь период (с 2017 по 2021): 2.85 %


In [92]:
# Экономическая эффективность определяется по формуле
# E(базовой модели) = PM/DM - (PMV + DY) / 365 = 5,6/39,37 - (10,42 + 5,27)/365 =  0,142- 0,042 = 0,1% в день
econom_effect_standart = round( (10.42 + 5.27)/365 ,2)
econom_effect_div = round(mean_prc_profit_div/days_trend,2)
print("Среднедневная доходность “Купи и держи” составляет:", econom_effect_standart, "%") 
print("Среднедневная доходность Базовой модели составляет:", econom_effect_div, "%") 
# По правилам выбранной в проекте оценки экономической эффективности 
# print("Экономическая эффективность Базовой модели относительно стандартной составляет:", econom_effect_div - econom_effect_standart, "% среднедневной доходности") 


Среднедневная доходность “Купи и держи” составляет: 0.04 %
Среднедневная доходность Базовой модели составляет: 0.04 %


In [159]:
 joblib.dump(model, '/content/drive/MyDrive/MTS.Project/Strategy/Trend_classification/strength_classification_model_GAZP.pkl')

['/content/drive/MyDrive/MTS.Project/Strategy/Trend_classification/strength_classification_model_GAZP.pkl']

# Добавление символов

In [165]:
symbols = ['LKOH', 'NVTK', 'ROSN', 'SIBN', 'SNGS',]
for symbol in symbols:
  df = pd.read_csv('/content/'+symbol+'_170101_210829.csv')
  df['<TIME>'][df['<TIME>']==0]='000000'
  df ['<DATE>'] = df['<DATE>'].astype(str)+df['<TIME>'].astype(str)
  df['<DATE>'] = pd.to_datetime(df['<DATE>'], format='%Y%m%d%H%M%S') 
  df.drop(columns=['<TIME>'], inplace=True)
  df_dif = df[1:].reset_index(drop = True)
  df_dif['<CLOSE>'] = df[1:]['<CLOSE>'].reset_index(drop = True,) - df[0:len(df)-1]['<CLOSE>']
  df_dif = df_dif[['<DATE>','<CLOSE>']]
  df_dif.columns = ['TIMEDATE', 'Close']
  #print (df_dif.head(1))
  trend_mark(df_dif)
  df_dif['trend_strength'] = trend_mark(df_dif)

  df_new = make_dataset(df_dif)
  print(df_new.head())

  X_train, y_train = (df_new.drop(columns = ['start_datatime','end_datatime','target_datetime', 'target',]), df_new['target'])
  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train_scaled = scaler.transform(X_train)
  
  #model  = OneVsRestClassifier(AdaBoostClassifier(n_estimators = 88, random_state=0)).fit(X_train_scaled, y_train)

  joblib.dump(scaler, '/content/drive/MyDrive/MTS.Project/Strategy/Trend_classification/Normalizer/strength_classification_normalizer_'+symbol+'.pkl')

  # preds = (model.predict(X_train)>0.5).astype(int)
  # print (make_score(y_train, preds))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|██████████| 12083/12083 [00:00<00:00, 1657285.00it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


       start_datatime  1dif  ...  target     target_datetime
0 2017-01-03 17:00:00   7.5  ...       2 2017-01-04 18:00:00
1 2017-01-04 19:00:00   7.0  ...       1 2017-01-06 11:00:00
2 2017-01-06 12:00:00 -35.5  ...       1 2017-01-09 13:00:00
3 2017-01-09 14:00:00   0.5  ...       1 2017-01-10 15:00:00
4 2017-01-10 16:00:00   9.5  ...       1 2017-01-11 17:00:00

[5 rows x 14 columns]


100%|██████████| 12083/12083 [00:00<00:00, 1709843.97it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


       start_datatime  1dif  ...  target     target_datetime
0 2017-01-03 17:00:00  -0.8  ...       1 2017-01-04 18:00:00
1 2017-01-04 19:00:00  -1.1  ...       1 2017-01-06 11:00:00
2 2017-01-06 12:00:00  -5.4  ...       1 2017-01-09 13:00:00
3 2017-01-09 14:00:00   3.9  ...       1 2017-01-10 15:00:00
4 2017-01-10 16:00:00   4.7  ...       1 2017-01-11 17:00:00

[5 rows x 14 columns]


100%|██████████| 12083/12083 [00:00<00:00, 2205290.25it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


       start_datatime  1dif  ...  target     target_datetime
0 2017-01-03 17:00:00 -0.30  ...       1 2017-01-04 18:00:00
1 2017-01-04 19:00:00  0.65  ...       2 2017-01-06 11:00:00
2 2017-01-06 12:00:00 -5.30  ...       2 2017-01-09 13:00:00
3 2017-01-09 14:00:00 -0.80  ...       1 2017-01-10 15:00:00
4 2017-01-10 16:00:00  1.45  ...       1 2017-01-11 17:00:00

[5 rows x 14 columns]


100%|██████████| 10573/10573 [00:00<00:00, 1997044.77it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


       start_datatime  1dif  ...  target     target_datetime
0 2017-01-03 14:00:00  0.15  ...       1 2017-01-04 15:00:00
1 2017-01-04 16:00:00  0.60  ...       0 2017-01-05 17:00:00
2 2017-01-05 18:00:00  0.25  ...       2 2017-01-06 19:00:00
3 2017-01-09 11:00:00  1.45  ...       1 2017-01-10 12:00:00
4 2017-01-10 13:00:00 -0.55  ...       1 2017-01-11 14:00:00

[5 rows x 14 columns]


100%|██████████| 12083/12083 [00:00<00:00, 2094638.36it/s]


       start_datatime   1dif  ...  target     target_datetime
0 2017-01-03 17:00:00 -0.035  ...       2 2017-01-04 18:00:00
1 2017-01-04 19:00:00  0.120  ...       1 2017-01-06 11:00:00
2 2017-01-06 12:00:00 -0.205  ...       1 2017-01-09 13:00:00
3 2017-01-09 14:00:00  0.015  ...       0 2017-01-10 15:00:00
4 2017-01-10 16:00:00  0.015  ...       1 2017-01-11 17:00:00

[5 rows x 14 columns]


In [169]:
df_new.tail(1)

,start_datatime,1dif,2dif,3dif,4dif,5dif,6dif,7dif,8dif,9dif,10dif,end_datatime,target,target_datetime
1097,2021-08-27 14:00:00,-0.025,0.005,-0.115,0.165,0.19,0.17,-0.09,-0.035,-0.035,0.05,2021-08-27 23:00:00,1,2021-08-28
